In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import ndjson
import copy 

def gendata():    
    for obj in data:
        if 'event' in obj:
            if 'options' in obj['event']:
                if type(obj['event']['options']) is list:
                    obj['event']['arr_options'] =copy.deepcopy(obj['event']['options'])   
                    del obj['event']['options']
            
            if type(obj['event']) is str:                
                obj['str_event'] = obj.pop('event') 
            elif type(obj['event']) is list:
                obj['list_event'] = obj.pop('event')
                
            
        
        
        yield {
            "_index": "online_courses",
            "_type": "doc",
            "_source": obj
        }
        
settings={
    "index.mapping.total_fields.limit": 10000
}

es = Elasticsearch([{"host": "localhost", "port": 9200}])

if es.indices.exists("online_courses"):
    es.indices.delete("online_courses")

es.indices.create(index="online_courses", body=settings)

with open("data/spbu.json", "r") as read_file:
    data = ndjson.load(read_file)
    bulk(es, gendata())

In [ ]:
"""

settings = {
    "mappings": {
        "doc": {
            "properties": {
                "event": {
                    "type": "object",
                    "properties":{
                        "cohort_id":{"type":"long"},
                        "user_id":{"type":"long"},
                        "cohort_name":{"type":"text"}
                    }
                }
            }
        }
     }
}

"settings": {
    "index.mapping.total_fields.limit": 2000,
    "number_of_shards": 1,
    "number_of_replicas": 0
  },
# create index
if es.indices.exists("test"):
    es.indices.delete("test")
es.indices.create(index="test", body=settings)
"""